# Grouping countries based on the Human Development Index (HDI)

In [1]:
import pandas as pd
import warnings
import functions

warnings.filterwarnings(action='ignore')

In [2]:
def get_hdi(path,filename):
    '''This function creates two dictionaries based on the Human Development Index and its levels.'''
    
    # Read in the xlsx-file with data
    file = "{}/{}".format(path,filename)
    hdi_data = pd.read_excel(file, header=None)
    
    # Replace the country names in the hdi_data with the corresponding country names as used in
    # the regions data frame.
    hdi_data[1] = hdi_data[1].replace(to_replace ="Viet Nam", value ="Vietnam") 
    hdi_data[1] = hdi_data[1].replace(to_replace ="Czechia", value ="Czech Republic") 
    hdi_data[1] = hdi_data[1].replace(to_replace ="Russian Federation", value ="Russia") 
    hdi_data[1] = hdi_data[1].replace(to_replace ="Venezuela (Bolivarian Republic of)", value ="Venezuela") 
    hdi_data[1] = hdi_data[1].replace(to_replace ="Korea (Republic of)", value ="South Korea") 
    hdi_data[1] = hdi_data[1].replace(to_replace ="Bolivia (Plurinational State of)", value ="Bolivia") 
    hdi_data[1] = hdi_data[1].replace(to_replace ="Hong Kong, China (SAR)", value ="Hong Kong") 
    hdi_data[1] = hdi_data[1].replace(to_replace ="Moldova (Republic of)", value ="Moldova") 
    hdi_data[1] = hdi_data[1].replace(to_replace ="Tanzania (United Republic of)", value ="Tanzania") 
    hdi_data[1] = hdi_data[1].replace(to_replace ="Lao People's Democratic Republic", value ="Laos") 
    hdi_data[1] = hdi_data[1].replace(to_replace ="Congo (Democratic Republic of the)", value ="Democratic Republic of the Congo") 
    hdi_data[1] = hdi_data[1].replace(to_replace ="American Samoa", value ="Samoa")
    
    # Select the useful rows and columns from the hdi data file to make the hdi.
    df_hdi = hdi_data.iloc[7:200, 1:3]
    # Remove the title rows indicating the human development level.
    df_hdi = df_hdi[df_hdi[2].notna()]
    # Make a dictionary with countries as keys and the hdi as values.
    dict_hdi = dict(df_hdi.values.tolist())
    
    # Select the useful rows and column from the hdi data file to make the hdi-levels.
    df_levels = hdi_data.iloc[7:200, [1]]
    # Create an index based on the title rows indicating the hdi-level.
    idx = df_levels[(df_levels[1].str.contains("HUMAN DEVELOPMENT"))].index
    # Use the index to create new dataframes per hdi-level.
    df_very_high = df_levels.iloc[idx[0]-6:idx[1]-7, :]
    df_high = df_levels.iloc[idx[1]-6:idx[2]-7, :]
    df_medium = df_levels.iloc[idx[2]-6:idx[3]-7, :]
    df_low = df_levels.iloc[idx[3]-6:, :]
    # Add a column with the hdi-level per data frame.
    df_very_high[2] = "very high"
    df_high[2] = "high"
    df_medium[2] = "medium"
    df_low[2] = "low"
    # Concatenate dataframes.
    df_hdi_levels = pd.concat([df_very_high, df_high, df_medium, df_low])
    # Make a dictionary with countries as keys and hdi-levels as values.
    dict_hdi_levels = dict(df_hdi_levels.values.tolist())
    
    print('Creating dictionaries for hdi and hdi-levels completed.')
    return dict_hdi, dict_hdi_levels

In [3]:
def create_hdi_columns(regions, dict_hdi, dict_hdi_levels):
    '''This function uses the hdi dictionary and the hdi-levels dictionary to create a column
    in the regions table with the hdi and the hdi-levels.'''
    
    # Create empty lists for hdi and hdi levels
    index = []
    levels = []
    
    # Get the hdi values for the countries in the regions dataframe and append it to the list.
    for i in range(len(regions)):  
        hd_idx = dict_hdi.get(regions.at[regions.index[i],"country_agg"],"NaN")
        index.append(hd_idx)
    print('Creating hdi list completed.')
    
    # Get the hdi index for the countries in the regions dataframe and append it to the list.
    for i in range(len(regions)):  
        hdi_lvls = dict_hdi_levels.get(regions.at[regions.index[i],"country_agg"],"NaN")
        levels.append(hdi_lvls)
    print('Creating hdi-level list completed.')
    
    # Fill the appended lists into their corresponding columns.
    regions["hdi"] = index
    regions["hdi_level"] = levels
    
    return regions

In [4]:
# Reading in data files from csv into a dictionary of dataframes.
dfs_region = functions.get_data("data/CMU_Global_data/Full_Survey_Data/region/smooth/", "region")

# Concatenating individuals dataframes from the dictionary into one dataframe for regions.
regions = pd.concat(dfs_region)

Read in data completed.


In [5]:
# Getting the dictionaries for the hdi and the hdi-levels.
dict_hdi_index, dict_hdi_levels = get_hdi("data","hdro_statistical_data_tables_1_15_d1_d5.xlsx")

# Creating a new column with the hdi and the hdi-levels in the regions dataframe
regions = create_hdi_columns(regions, dict_hdi_index, dict_hdi_levels)

In [6]:
regions[["country_agg","hdi","hdi_level"]].head(30)

country_agg       hdi hdi_level
region_05 0   Afghanistan   0.49596       low
          1   Afghanistan   0.49596       low
          2   Afghanistan   0.49596       low
          3   Afghanistan   0.49596       low
          4       Albania  0.791406      high
          5       Albania  0.791406      high
          6       Albania  0.791406      high
          7       Albania  0.791406      high
          8       Albania  0.791406      high
          9       Algeria  0.758978      high
          10      Algeria  0.758978      high
          11      Algeria  0.758978      high
          12      Algeria  0.758978      high
          13      Algeria  0.758978      high
          14      Algeria  0.758978      high
          15      Algeria  0.758978      high
          16      Algeria  0.758978      high
          17      Algeria  0.758978      high
          18      Algeria  0.758978      high
          19      Algeria  0.758978      high
          20      Algeria  0.758978      high
          21      Algeria  0.758978      high
          22      Algeria  0.758978      high
          23      Algeria  0.758978      high
          24      Algeria  0.758978      high
          25      Algeria  0.758978      high
          26      Algeria  0.758978      high
          27      Algeria  0.758978      high
          28      Algeria  0.758978      high
          29      Algeria  0.758978      high

### On checking differences in names

There were many NaN's when looking up the country in the dictionary that could be explained by differences in country names. This was checked below. The solution is already included in the get hdi function.

In [ ]:
check = regions[["country_agg","hdi_index"]]
check[check["hdi_index"] == "NaN"].country_agg.value_counts()

### Differences in naming regions and hdi

regions|hdi
---|---
Vietnam|Viet Nam
Czech Republic|Czechia
Russia|Russian Federation
Venezuela|Venezuela (Bolivarian Republic of)
South Korea|Korea (Republic of)
Bolivia|Bolivia (Plurinational State of)
Hong Kong|Hong Kong, China (SAR)
Taiwan|*non-existent*
Moldova|Moldova (Republic of)
Tanzania|Tanzania (United Republic of)
Laos|Lao People's Democratic Republic
Democratic Republic of the Congo|Congo (Democratic Republic of the)
American Samoa|Samoa

In [ ]:
data = pd.read_excel("data/hdro_statistical_data_tables_1_15_d1_d5.xlsx", header=None)

In [ ]:
data[1] = data[1].replace(to_replace ="Viet Nam", value ="Vietnam") 
data[1] = data[1].replace(to_replace ="Czechia", value ="Czech Republic") 
data[1] = data[1].replace(to_replace ="Russian Federation", value ="Russia") 
data[1] = data[1].replace(to_replace ="Venezuela (Bolivarian Republic of)", value ="Venezuela") 
data[1] = data[1].replace(to_replace ="Korea (Republic of)", value ="South Korea") 
data[1] = data[1].replace(to_replace ="Bolivia (Plurinational State of)", value ="Bolivia") 
data[1] = data[1].replace(to_replace ="Hong Kong, China (SAR)", value ="Hong Kong") 
data[1] = data[1].replace(to_replace ="Moldova (Republic of)", value ="Moldova") 
data[1] = data[1].replace(to_replace ="Tanzania (United Republic of)", value ="Tanzania") 
data[1] = data[1].replace(to_replace ="Lao People's Democratic Republic", value ="Laos") 
data[1] = data[1].replace(to_replace ="Congo (Democratic Republic of the)", value ="Democratic Republic of the Congo") 
data[1] = data[1].replace(to_replace ="American Samoa", value ="Samoa")